# Day 3 - Lab 1: AI-Driven Backend Development

**Objective:** Generate a complete FastAPI backend application, including Pydantic and SQLAlchemy models, and then perform the critical engineering task of integrating the generated code with the live SQLite database created on Day 2.

**Estimated Time:** 135 minutes

**Introduction:**
Welcome to Day 3! With our requirements and architecture defined, it's time to write code. In this lab, you will act as a senior developer guiding an AI co-pilot. Your task is to generate the full backend API for the Onboarding Tool. This involves not just generating code, but also connecting it to the live database we created yesterday, moving from a prototype to a functional, data-driven application.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We'll set up our environment and load the `schema.sql` artifact from Day 2. This SQL file contains the `CREATE TABLE` statements that define our database structure, which is the perfect context to provide the LLM for code generation.

**Model Selection:**
For code generation, models specifically fine-tuned for coding are ideal. `gpt-4.1`, `o3`, or `codex-mini` are excellent choices. Experiment to see which one gives you the cleanest code.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the SQL schema.
- `save_artifact()`: To save the generated Python code.
- `clean_llm_output()`: To remove markdown fences from the generated code.

In [1]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

# Load the SQL schema from Day 2
sql_schema = load_artifact("artifacts/schema.sql")
if not sql_schema:
    print("Warning: Could not load schema.sql. Lab may not function correctly.")

2025-10-28 15:34:12,940 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


## Step 2: The Challenges

Follow the challenges below to build and connect your API.

### Challenge 1 (Foundational): Generating Code with In-Memory Logic

**Task:** Generate all the necessary Python code for a FastAPI application, but with simple in-memory data storage for now. This allows us to generate and validate the code's structure before adding database complexity.

**Instructions:**
1.  Create a detailed prompt that asks the LLM to act as a senior Python developer.
2.  Provide the `sql_schema` as context.
3.  Instruct the LLM to generate three key components:
    * **Pydantic Models:** For API data validation (request/response bodies).
    * **FastAPI Endpoints:** Full CRUD (Create, Read, Update, Delete) endpoints for the `users` table.
    * **In-Memory Database:** A simple Python list to act as a temporary, fake database.
4.  The final output should be a single Python script for a `main_in_memory.py` file.
5.  Save the generated code to `app/main_in_memory.py`.

In [2]:
# TODO: Write a prompt to generate a complete FastAPI application with in-memory data storage.
in_memory_api_prompt = f"""
You are a senior Python developer with expertise in FastAPI, Pydantic, and modern Python development practices. Your task is to create a complete FastAPI application for an employee onboarding system.

## Database Schema Context:
{sql_schema}

## Requirements:

1. **Create Pydantic Models**: Design comprehensive Pydantic models for data validation that correspond to the database schema. Include:
   - Request models for creating new records (without auto-generated fields like IDs)
   - Response models for API responses (with all fields including IDs and timestamps)
   - Update models for partial updates (using Optional fields)
   - Proper field validation, constraints, and documentation

2. **Implement FastAPI Application**: Create a complete FastAPI app with:
   - Proper imports and app initialization
   - Comprehensive CRUD endpoints for the users table
   - Proper HTTP status codes and error handling
   - Detailed endpoint documentation with descriptions and examples
   - Input validation using Pydantic models
   - Response models for consistent API responses

3. **In-Memory Database**: Implement a simple in-memory storage system using Python lists/dictionaries that simulates database operations:
   - Store users in a list with auto-incrementing IDs
   - Implement proper data persistence during the application lifecycle
   - Handle edge cases like duplicate emails, missing records, etc.

## Specific Endpoints Required:

- `POST /users/` - Create a new user
- `GET /users/` - List all users (with optional pagination)
- `GET /users/{{user_id}}` - Get a specific user by ID
- `PUT /users/{{user_id}}` - Update a user (full update)
- `PATCH /users/{{user_id}}` - Partial update of a user
- `DELETE /users/{{user_id}}` - Delete a user

## Code Quality Standards:

- Use type hints throughout
- Include comprehensive docstrings
- Follow PEP 8 style guidelines
- Implement proper error handling with meaningful error messages
- Use FastAPI's automatic OpenAPI documentation features
- Include example data in the documentation
- Make the code production-ready with proper logging considerations

## Output Format:

Provide a single, complete Python file that can be run directly. The file should be named `main_in_memory.py` and should include:
- All necessary imports
- Pydantic model definitions
- FastAPI app initialization
- All endpoint implementations
- In-memory database simulation
- Proper main block for running the application

The code should be immediately runnable with `python main_in_memory.py` and should start a FastAPI server on the default port.

Generate clean, well-structured, and thoroughly documented code that demonstrates best practices for FastAPI development.
"""

print("--- Generating FastAPI app with in-memory database ---")
if sql_schema:
    generated_api_code = get_completion(in_memory_api_prompt, client, model_name, api_provider)
    cleaned_code = clean_llm_output(generated_api_code, language='python')
    print(cleaned_code)
    save_artifact(cleaned_code, "app/main_in_memory.py")
else:
    print("Skipping API generation because schema is missing.")

--- Generating FastAPI app with in-memory database ---
# main_in_memory.py

"""
A complete FastAPI application for an employee onboarding system.

This application provides a RESTful API for managing users within the system.
It uses an in-memory dictionary to simulate a database, making it a self-contained
and runnable example.

Features:
- Pydantic models for robust data validation and serialization.
- Full CRUD (Create, Read, Update, Delete) functionality for users.
- In-memory data storage that persists for the application's lifecycle.
- Detailed OpenAPI/Swagger documentation generated automatically by FastAPI.
- Best practices including type hints, dependency injection, and proper error handling.

To run this application:
1. Make sure you have the required packages installed:
   pip install "fastapi[all]"
2. Run the script from your terminal:
   python main_in_memory.py
3. The API will be available at http://127.0.0.1:8000
4. Interactive API documentation (Swagger UI) will be at ht

### Challenge 2 (Intermediate): Generating Database Models and Session Code

**Task:** Now, generate the specific SQLAlchemy code required to connect our application to the live `onboarding.db` SQLite database.

**Instructions:**
1.  Create a new prompt.
2.  Provide the `sql_schema` as context again.
3.  Instruct the LLM to generate two separate pieces of code:
    * **SQLAlchemy Models:** Python classes that map to your database tables.
    * **Database Session Management:** The boilerplate code to create a database engine, session maker, and a dependency function (`get_db`) for use in FastAPI.
4.  The output should be two distinct, well-commented Python code blocks. We will integrate these manually in the next step.

In [3]:
# TODO: Write a prompt to generate SQLAlchemy models and the database session/dependency code.
db_code_prompt = f"""
You are a senior Python developer with expertise in SQLAlchemy, FastAPI, and database design. Your task is to create SQLAlchemy models and database session management code for an employee onboarding system.

## Database Schema Context:
{sql_schema}

## Requirements:

### Part 1: SQLAlchemy Models
Create comprehensive SQLAlchemy model classes that map to the database tables. For each model:

1. **Use SQLAlchemy 2.0+ syntax** with `DeclarativeBase` and modern column definitions
2. **Include all table columns** with proper data types and constraints
3. **Implement relationships** between tables using `relationship()` and `ForeignKey()`
4. **Add proper constraints** including CHECK constraints, UNIQUE constraints, and NOT NULL constraints
5. **Include timestamps** for created_at and updated_at fields where applicable
6. **Use proper naming conventions** following Python standards
7. **Add comprehensive docstrings** explaining each model and its relationships
8. **Include example data** in docstrings where helpful

### Part 2: Database Session Management
Create the database connection and session management infrastructure:

1. **Database Engine Configuration**: Set up SQLAlchemy engine for SQLite database (`onboarding.db`)
2. **Session Factory**: Create a session maker with proper configuration
3. **FastAPI Dependency**: Implement a `get_db()` dependency function for FastAPI
4. **Connection Management**: Include proper connection pooling and cleanup
5. **Error Handling**: Add appropriate error handling for database operations
6. **Configuration**: Make database URL configurable via environment variables

## Specific Requirements:

### Models to Create:
- `OnboardingPath` (maps to Onboarding_Paths)
- `User` (maps to Users) 
- `OnboardingTask` (maps to Onboarding_Tasks)
- `PathTask` (maps to Path_Tasks)
- `UserTaskStatus` (maps to User_Task_Status)
- `Document` (maps to Documents)
- `UserDocument` (maps to User_Documents)
- `KBCategory` (maps to KB_Categories)
- `KBArticle` (maps to KB_Articles)
- `KBTag` (maps to KB_Tags)
- `ArticleTag` (maps to Article_Tags)

### Key Relationships to Implement:
- User -> Mentor (self-referential)
- User -> OnboardingPath
- OnboardingPath -> OnboardingTasks (many-to-many via PathTasks)
- User -> TaskStatus (many-to-many via UserTaskStatus)
- User -> Documents (many-to-many via UserDocuments)
- KBArticle -> Author (User)
- KBArticle -> Category
- KBArticle -> Tags (many-to-many via ArticleTags)

### Session Management Features:
- Database engine with SQLite configuration
- Session maker with proper settings
- FastAPI dependency function `get_db()`
- Proper session lifecycle management
- Environment variable support for database URL
- Connection pooling configuration

## Code Quality Standards:

- Use SQLAlchemy 2.0+ declarative syntax
- Include comprehensive type hints
- Add detailed docstrings for all classes and methods
- Follow PEP 8 style guidelines
- Implement proper error handling
- Use modern Python features (dataclasses, enums where appropriate)
- Include configuration management
- Make code production-ready with proper logging considerations

## Output Format:

Provide two distinct, well-commented Python code blocks:

**Block 1: SQLAlchemy Models**
```python
# SQLAlchemy Models
# Include all model classes with relationships
```

**Block 2: Database Session Management**
```python
# Database Session Management
# Include engine, session maker, and FastAPI dependency
```

Each block should be complete and ready to integrate into a FastAPI application. The models should handle all the complex relationships in the schema, and the session management should be robust and production-ready.

Generate clean, well-structured, and thoroughly documented code that demonstrates best practices for SQLAlchemy and FastAPI integration.
"""

print("--- Generating SQLAlchemy Models and Session Code ---")
if sql_schema:
    generated_db_code = get_completion(db_code_prompt, client, model_name, api_provider)
    print("\n--- Generated Database Code ---")
    print(generated_db_code)
else:
    print("Skipping DB code generation because schema is missing.")

--- Generating SQLAlchemy Models and Session Code ---

--- Generated Database Code ---
Of course. As a senior Python developer, I will provide a comprehensive and production-ready solution for your employee onboarding system, adhering to all best practices for SQLAlchemy, FastAPI, and modern Python development.

Here are the two requested code blocks, meticulously crafted to meet your requirements.

***

### Block 1: SQLAlchemy Models

This block contains the SQLAlchemy models that map to your database schema. It utilizes SQLAlchemy 2.0 syntax, includes all relationships, constraints, and is thoroughly documented. Enums are used for `CHECK` constraints to improve code readability and maintainability.

```python
# SQLAlchemy Models for the Employee Onboarding System
# File: models.py

import enum
from datetime import date, datetime
from typing import List, Optional, Set

from sqlalchemy import (
    CheckConstraint,
    Column,
    Date,
    DateTime,
    ForeignKey,
    Integer,
    St

### Challenge 3 (Advanced): Integrating Live Database Logic

**Task:** This is the most critical engineering step of the lab. You will manually integrate the generated database code into the FastAPI application, replacing the in-memory logic with live database operations.

**Instructions:**
This task represents a significant jump in complexity. Follow these steps carefully in your IDE (like VS Code):

1.  Create a new, empty file named `app/main.py`.
2.  **First, copy the Pydantic models and the `app = FastAPI()` line** from your `app/main_in_memory.py` file and paste them into `app/main.py`.
3.  **Next, paste the SQLAlchemy model classes and the `get_db` dependency function** you generated in Challenge 2 into your new `app/main.py`.
4.  **Now, let's refactor the `POST /users/` endpoint.** Copy the endpoint function from the in-memory file, but replace the in-memory logic (e.g., `db.append()`) with the correct SQLAlchemy session calls: `db.add(db_user)`, `db.commit()`, and `db.refresh(db_user)`.
5.  Repeat this refactoring process for the other endpoints (GET, PUT, DELETE), replacing list manipulations with the appropriate SQLAlchemy `db.query()` methods.

This task requires you to act as the senior developer, stitching together the AI-generated components into a functional, cohesive whole. You may need to ask the LLM follow-up questions like, "How do I write a SQLAlchemy query to find a user by ID?"

## Lab Conclusion

Congratulations! You have successfully generated and assembled a complete, database-connected backend API. You used an LLM to generate the boilerplate for both the API endpoints and the database models, and then performed the crucial engineering task of integrating them. You now have a working `main.py` file in your `app` directory that can create, read, update, and delete data in a live database. In the next lab, we will write a comprehensive test suite for this API.

> **Key Takeaway:** AI excels at generating boilerplate code (like models and endpoint structures), but the developer's critical role is in the final integration and wiring of these components into a coherent, working system.